In [1]:
import fasttext
import re  #正则表达式
import pandas as pd

In [3]:
train = pd.read_table('data/train.tsv', names=['label', 'content'], encoding='utf-8')
# train.head(5)

In [4]:
valid = pd.read_table('data/val.tsv', names=['label', 'content'], encoding='utf-8')

In [5]:
def label_string(label):
    labs = ''
    for lab in label.split('@'):
        labs += "\t__label__" + str(lab)
    return labs


def single_label_string(label, idx):
    "idx: 0 - 19"
    return "\t__label__" + str(label.split('@')[idx])

In [11]:
def to_file(file, x, y, label_func):
    
    for i, line in enumerate(x):
        outline = line + label_func(y[i]) + "\n"
        file.write(outline)
        if i % 30000 == 0:
            print(outline)
    file.close()

### 效果一般方式：20种标签联合多分类

In [12]:
ftrain = open("data/sentiment_train.txt", "w", encoding='utf-8')
ftest = open("data/sentiment_test.txt", "w", encoding='utf-8')

to_file(ftrain, train.content, train.label, label_string)
to_file(ftest, valid.content, valid.label, label_string)

吼吼吼～萌死人的棒棒糖～中了大众点评的霸王餐～太可爱了。一直就好奇这个棒棒糖是怎么个东西～大众点评给了我这个土老冒一个见识的机会。看介绍棒棒糖是用德国糖做的～不会很甜～中间的照片是糯米的～能食用～真是太高端大气上档次了～还可以买蝴蝶结扎口～送人可以买礼盒。我是先打的卖家电话～加了微信～给卖家传的照片。等了几天～卖家就告诉我可以取货了～去大官屯那取的。虽然连卖家的面都没见到～但是还是谢谢卖家送我这么可爱的东西～太喜欢了～这哪舍得吃啊。	__label__0_-2	__label__1_-2	__label__2_-2	__label__3_-2	__label__4_1	__label__5_-2	__label__6_-2	__label__7_-2	__label__8_-2	__label__9_1	__label__10_-2	__label__11_-2	__label__12_-2	__label__13_-2	__label__14_-2	__label__15_-2	__label__16_1	__label__17_-2	__label__18_1	__label__19_-2

一直很喜欢这家的pasta～以前都是在美罗城楼下吃的～感觉都很好还一直和别人推荐～周日下雨和老公本来想去太兴的没想到关门就到镰仓了～这里地方还算比较大～一进去浓浓的奶香味很舒服～坐下来开始点单～新品出来还比较多～不过还是点了pasta。畅食的面包必须点单后才给你上ok没问题～我点了脆虾番茄奶油意面52元～虾大概有6-8个还算可以～不过意面不多～最重要的是这里的意面能叫意面吗？烂的比烂糊面还烂～加上烧的浓稠的的酱汁要死了就是一坨面粉～完全没有口味可言～老公点的扇贝配鲑鱼意面也是如此～面条都糊了～受不了～吃完胃就不舒服了。熏鲑鱼色拉还可以酱料调配的比较爽口。畅食的面包真的没有以前好吃了～以前的羊角面包都是现烤热腾腾的～现在很多都是一起上～虽然味道还不错但是口感没有刚出炉的那么酥脆了。看来以后老公是不会再来了。	__label__0_-2	__label__1_-2	__label__2_-2	__label__3_-2	__label__4_-2	__label__5_-2	__label__6_-2	__label__7_0	__label__8_-2	__label__9_

In [8]:
from fasttext import train_supervised

In [28]:
model = train_supervised(
    input="data/sentiment_train.txt",
    epoch=50,
    lr=0.2,
    dim=300,
    loss='ova',
    wordNgrams=2,
    verbose=1,
    minCount=1)

In [29]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))
    print("f1@ \t {:.3f}".format(2 * p * r / (p + r)))

In [32]:
# ova loss
print_results(*model.test("data/sentiment_test.txt", k=20))

N	15000
P@1	0.679
R@1	0.679
f1@ 	 0.679


In [27]:
# softmax loss
print_results(*model.test("data/sentiment_test.txt", k=20))

N	15000
P@1	0.683
R@1	0.683
f1@ 	 0.683


In [48]:
del model

### 每种标签分开计算

In [34]:
with open('labels.txt',  'r', encoding='utf-8') as f:
    labels = f.readlines()
    
print(labels)

# def process_label(pandas_df):
#     pandas_df = pd.concat([pandas_df, pd.DataFrame(columns=[i for i in range(20)])])
#     for idx, row in pandas_df.iterrows():
#         label = row.label.split('@')
#         for i, la in enumerate(label):
#             row[i] = la
#     return pandas_df

# train_pro = process_label(train)

# valid_pro = process_label(valid)

['location_traffic_convenience\n', 'location_distance_from_business_district\n', 'location_easy_to_find\n', 'service_wait_time\n', 'service_waiters_attitude\n', 'service_parking_convenience\n', 'service_serving_speed\n', 'price_level\n', 'price_cost_effective\n', 'price_discount\n', 'environment_decoration\n', 'environment_noise\n', 'environment_space\n', 'environment_cleaness\n', 'dish_portion\n', 'dish_taste\n', 'dish_look\n', 'dish_recommendation\n', 'others_overall_experience\n', 'others_willing_to_consume_again\n']


- preprocessing the data ;
- changing the number of epochs (using the option -epoch, standard range [5 - 50]) ;
- changing the learning rate (using the option -lr, standard range [0.1 - 1.0]) ;
- using word n-grams (using the option -wordNgrams, standard range [1 - 5]).

https://fasttext.cc/docs/en/python-module.html#api


In [44]:
from fasttext import train_supervised


def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))
    print("f1@ \t {:.3f}".format(2 * p * r / (p + r)))

In [45]:
def gen_train_file(label_idx):
    ftrain = open(f"data/sentiment_train_{label_idx}.txt", "w", encoding='utf-8')
    ftest = open(f"data/sentiment_test_{label_idx}.txt", "w", encoding='utf-8')

    to_file(ftrain, train.content, train.label, lambda label: single_label_string(label, label_idx))
    to_file(ftest, valid.content, valid.label, lambda label: single_label_string(label, label_idx))

In [47]:
gen_train_file(1)

吼吼吼～萌死人的棒棒糖～中了大众点评的霸王餐～太可爱了。一直就好奇这个棒棒糖是怎么个东西～大众点评给了我这个土老冒一个见识的机会。看介绍棒棒糖是用德国糖做的～不会很甜～中间的照片是糯米的～能食用～真是太高端大气上档次了～还可以买蝴蝶结扎口～送人可以买礼盒。我是先打的卖家电话～加了微信～给卖家传的照片。等了几天～卖家就告诉我可以取货了～去大官屯那取的。虽然连卖家的面都没见到～但是还是谢谢卖家送我这么可爱的东西～太喜欢了～这哪舍得吃啊。	__label__1_-2

一直很喜欢这家的pasta～以前都是在美罗城楼下吃的～感觉都很好还一直和别人推荐～周日下雨和老公本来想去太兴的没想到关门就到镰仓了～这里地方还算比较大～一进去浓浓的奶香味很舒服～坐下来开始点单～新品出来还比较多～不过还是点了pasta。畅食的面包必须点单后才给你上ok没问题～我点了脆虾番茄奶油意面52元～虾大概有6-8个还算可以～不过意面不多～最重要的是这里的意面能叫意面吗？烂的比烂糊面还烂～加上烧的浓稠的的酱汁要死了就是一坨面粉～完全没有口味可言～老公点的扇贝配鲑鱼意面也是如此～面条都糊了～受不了～吃完胃就不舒服了。熏鲑鱼色拉还可以酱料调配的比较爽口。畅食的面包真的没有以前好吃了～以前的羊角面包都是现烤热腾腾的～现在很多都是一起上～虽然味道还不错但是口感没有刚出炉的那么酥脆了。看来以后老公是不会再来了。	__label__1_-2

本人上周参加了婚宴！灯光效果非常不错～做的挺用心。服务员态度比较谦和～也挺任劳任怨的～一个个东跑西癫都累出汗了～看着俺都心疼啊。用于闹肚子～喝点白酒也没吃菜～不过听老婆说个个菜做的超棒～非得中秋节拉着我去尝尝果木烤鸭和杏鲍菇。以后想吃海鲜就去奧帆号了～海鲜厨房嘛高端的装修～可口的大众亲民的价格。还有好多帅哥美女免费服务哟～唯一美中不足的地方就是院子里的停车场有点小～放30多个车就快没地方了～得提前点过去抢个车位再说～哈哈	__label__1_-2

一～地理位置：谢家湾万象城5楼～溜冰场上面。二～服务：店铺不大～干净整洁。服务态度不咸不淡的～9积分兑换的～也可以理解三～食品：有一句话真的很对—不作死就不会死。本来不喜欢喝茶～但作为吃货～没尝过黑龙茶～觉得还是有点介意。刚好有9积分～就鼓起勇气尝试一下。要的常温的红茶～想到热量问题～少糖～也没有加配料。然后就悲剧了～茶很大一杯

In [52]:
def train_label(idx):
    model = train_supervised(
        input=f"data/sentiment_train_{idx}.txt",
        epoch=50,
        lr=0.5,
        dim=100,
        loss='softmax',
        wordNgrams=2,
        verbose=1,
        minCount=1)
    return model

In [53]:
model_1 = train_label(1)

In [56]:
print_results(*model_1.test('data/sentiment_test_1.txt'))

N	15000
P@1	0.796
R@1	0.796
f1@ 	 0.796


In [58]:
del model_1

### 超参数搜索模型

In [60]:
# hyperparameter optimization
# 最优模型超大

def search(idx):
    model = train_supervised(input=f"data/sentiment_train_{idx}.txt", 
                                             autotuneValidationFile=f"data/sentiment_test_{idx}.txt",
                                             autotuneMetric="f1",
                                             autotuneDuration=600,
                                             verbose=0)
    return model

In [57]:
def test_model(idx):
    print_results(*model.test(f"data/sentiment_test_{idx}.txt"))

In [ ]:
model_1 = search(1)

In [1]:
tp = 100
fp = 20
tn = 100
fn = 20

p = tp / (tp + fp)
r = tp / (tp + fn)

f1 = 2 * p * r / (p + r)

print(f1)

0.8333333333333334


In [5]:
tp = 10
fp = 7
tn = 10
fn = 7

p = tp / (tp + fp)
r = tp / (tp + fn)

f1 = 2 * p * r / (p + r)

print(f1)

0.5882352941176471
